In [1]:
import numpy as np
import pandas as pd
import warnings

In [2]:
%run function.ipynb

In [3]:
path = 'D:\\資料處理\\SBAdata\\SBAnational.csv'
df = pd.read_csv(path)
df_copy = df.copy()
display(df.iloc[:5,:11])
display(df.iloc[:5,11:])

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240


,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,2,1.0,0,0,1,0,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,14,1.0,7,7,1,0,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [4]:
df_copy.shape

(899164, 27)

In [5]:
df2 = df.iloc[:50, :]

In [6]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [7]:
# 處理日期
d_cols = ['ChgOffDate', 'DisbursementDate', 'ApprovalDate']
df_copy[d_cols] = df_copy[d_cols].fillna('xx-xx-xx')

for col in d_cols:
    
    df_copy[col + '_year'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[2])
    df_copy[col + '_month'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[1])
    df_copy[col + '_day'] = df_copy[col].astype('str').apply(lambda x: x.split('-')[0])

# df_copy.drop(d_cols, axis=1, inplace=True)
df_copy.drop('ApprovalFY', axis=1, inplace=True)

mon_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12, 'xx': 'xx'}

# 月份文字轉數字
df_copy['ChgOffDate_month'] = df_copy['ChgOffDate_month'].map(mon_dict)
df_copy['DisbursementDate_month'] = df_copy['DisbursementDate_month'].map(mon_dict)
df_copy['ApprovalDate_month'] = df_copy['ApprovalDate_month'].map(mon_dict)

year_cols = ['ChgOffDate_year', 'DisbursementDate_year', 'ApprovalDate_year']

df_copy[year_cols] = df_copy[year_cols].astype('str').applymap(year_2d_to_4d)

In [8]:
# 金錢欄位清理
money_cols = ['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
df_copy.loc[:, money_cols] = df_copy.loc[:, money_cols].applymap(money_clean)
df_copy[money_cols] = df_copy[money_cols].astype('int64')

In [9]:
# 類別欄位清理
df_copy['NewExist'] = df_copy['NewExist'].map({2: 0, 1: 1})
df_copy['UrbanRural'] = df_copy['UrbanRural'].map({2: 0, 0: -1, 1: 1})
df_copy['RevLineCr'] = df_copy['RevLineCr'].map({'Y': 1, 'N': 0})
df_copy['LowDoc'] = df_copy['LowDoc'].map({'Y': 1, 'N': 0})

In [10]:
df_copy['Industry'] = df_copy['NAICS'].astype('str').apply(lambda x: x[:2])
df_copy['Industry'] = df_copy['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin',
    '0': 'Unknown'
})

df_copy.drop('NAICS', axis=1, inplace=True)

In [11]:
df_copy = df_copy.apply(lambda x: x.replace('xx', np.nan))
df_copy = df_copy.apply(lambda x: x.replace('xx-xx-xx', np.nan))

# 缺失值處理

In [12]:
# 找不到 RevLineCr的缺失規律，填-1變第三個類別
df_copy['RevLineCr'].fillna(-1, inplace=True)
df_copy['RevLineCr'].value_counts()

 0.0    420288
-1.0    277479
 1.0    201397
Name: RevLineCr, dtype: int64

In [13]:
# 找不到 LowDoc的缺失規律，填-1變第三個類別
df_copy['LowDoc'].fillna(-1, inplace=True)
df_copy['LowDoc'].value_counts()

 0.0    782822
 1.0    110335
-1.0      6007
Name: LowDoc, dtype: int64

MIS_Status相關處理

In [14]:
# 兩者比例
df_copy['MIS_Status'].value_counts()
# 19  ChgOffDate  162699 non-null  object

P I F     739609
CHGOFF    157558
Name: MIS_Status, dtype: int64

In [15]:
na_check(df_copy)

,Missing Ratio
ChgOffDate,0.819055
ChgOffDate_year,0.819055
ChgOffDate_month,0.819055
ChgOffDate_day,0.819055
DisbursementDate,0.002634
DisbursementDate_year,0.002634
DisbursementDate_month,0.002634
DisbursementDate_day,0.002634
MIS_Status,0.002221
BankState,0.001742


In [16]:
# 把target不見的抽出來
df_target_miss = df_copy[is_na_or_not(df_copy['MIS_Status'])]
df_target_miss.head(5)
display(na_check(df_target_miss))

,Missing Ratio
MIS_Status,1.000000
ChgOffDate,0.869304
ChgOffDate_year,0.869304
ChgOffDate_month,0.869304
ChgOffDate_day,0.869304
DisbursementDate,0.096645
DisbursementDate_year,0.096645
DisbursementDate_month,0.096645
DisbursementDate_day,0.096645
Bank,0.026540


In [17]:
pif_increase = 0  # 計數
chg_increase = 0

# 先定義中位數
med = np.median(df_copy[(df_copy['ChgOffPrinGr'] != 0) & (pd.isnull(df_copy['ChgOffPrinGr']) == False)]['ChgOffPrinGr'])
print('median:', med)

for i in list(df_target_miss.index):
    # 有日期的 CHGOFF 沒日期的 PIF
    # 這邊也把金額改了不然後面會亂掉
    if pd.isnull(df_copy.loc[i, 'ChgOffDate']):
        df_copy.loc[i, 'MIS_Status'] = 'CHGOFF'
        chg_increase += 1
        df_copy.loc[i, 'ChgOffPrinGr'] = med
    else:
        df_copy.loc[i, 'MIS_Status'] = 'P I F'
        pif_increase += 1
        df_copy.loc[i, 'ChgOffPrinGr'] = 0

# 檢查null num
print('null num after operation:', df_copy['MIS_Status'].isnull().sum())
print('P I F increase:', pif_increase)
print('CHGOFF increase:', chg_increase)

median: 32455.5
null num after operation: 0
P I F increase: 261
CHGOFF increase: 1736


In [18]:
df_pif_weird = df_copy[((df_copy['MIS_Status'] == 'CHGOFF')&(df_copy['ChgOffPrinGr'] == 0)) | 
                       ((df_copy['MIS_Status'] == 'P I F')&(df_copy['ChgOffPrinGr'] != 0))]
df_pif_weird[['Name', 'ChgOffPrinGr', 'MIS_Status', 'FranchiseCode']].sort_values(by='Name', ascending=True).head(30)
df_pif_weird['Name'].value_counts()

AAMCO TRANSMISSIONS               3
GTP INC                           3
THE SWEATER COMPANY               3
SCOTT AVIATION                    3
ARMSTRONG PAINTING INC            2
                                 ..
GIRLS RULE SPORTS CORP            1
K & M HOME FASHIONS               1
THE LEONE CORPORATION             1
SHEYENNE RIVER ENTERPRISES,INC    1
VILLA CONSTRUCTION LLC            1
Name: Name, Length: 5489, dtype: int64

In [19]:
df_pif_weird[['Name', 'City', 'ChgOffPrinGr', 'MIS_Status', 'FranchiseCode', 'ApprovalDate', 'ApprovalDate_year', 'ChgOffDate', 'ChgOffDate_year']].head(5)

,Name,City,ChgOffPrinGr,MIS_Status,FranchiseCode,ApprovalDate,ApprovalDate_year,ChgOffDate,ChgOffDate_year
489,"BLOOMINGSHIRE, INC.",BLOOMINGDALE,0.0,CHGOFF,0,30-Jun-80,1980,16-May-90,1990
558,GREENLIFE MOBILE,SAN JOSE,3330.0,P I F,1,7-Feb-06,2006,19-Oct-09,2009
850,GILLIAN'S DELICATESSEN & CATER,FLORAL PARK,10270.0,P I F,1,26-Mar-03,2003,29-Mar-06,2006
853,TRICAS MANAGEMENT SERVICES LLC,DALLAS,97486.0,P I F,1,8-Feb-06,2006,2-Feb-09,2009
861,CAPITAL EXPLOSION GRAPHIC DESI,UPPER MARLBORO (MARLBORO),2310.0,P I F,1,8-Feb-06,2006,14-Aug-12,2012


In [20]:
df_pif_weird.ChgOffDate.isnull().sum()  # 決定性證據

47

In [21]:
# 這些 標籤改成 CHGOFF
df_pif_weird_pif = df_pif_weird[df_pif_weird['MIS_Status'] == 'P I F']
print('weird pif shape: ', df_pif_weird_pif.shape)
print('weird pif chgoffdate nullnum: ', df_pif_weird_pif.ChgOffDate.isnull().sum())
display(df_pif_weird_pif.head(5))

# 改標籤
df_copy.loc[list(df_pif_weird_pif.index), 'MIS_Status'] = 'CHGOFF'

print('check')
df_copy.loc[list(df_pif_weird_pif.index), 'MIS_Status'].value_counts()

weird pif shape:  (4884, 35)
weird pif chgoffdate nullnum:  0


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
558,1004176005,GREENLIFE MOBILE,SAN JOSE,CA,95132,BBCN BANK,CA,7-Feb-06,42,1,...,2009,10.0,19,2006,2.0,28,2006,2,7,Admin_sup/Waste_Mgmt_Rem
850,1005925000,GILLIAN'S DELICATESSEN & CATER,FLORAL PARK,NY,11001,JPMORGAN CHASE BANK NATL ASSOC,IL,26-Mar-03,13,1,...,2006,3.0,29,2003,3.0,31,2003,3,26,Accom/Food_serv
853,1005936010,TRICAS MANAGEMENT SERVICES LLC,DALLAS,TX,75218,WELLS FARGO BANK NATL ASSOC,SD,8-Feb-06,27,7,...,2009,2.0,2,2006,3.0,31,2006,2,8,Healthcare/Social_assist
861,1005986003,CAPITAL EXPLOSION GRAPHIC DESI,UPPER MARLBORO (MARLBORO),MD,20774,BBCN BANK,CA,8-Feb-06,7,3,...,2012,8.0,14,2006,2.0,28,2006,2,8,Prof/Science/Tech
866,1006016007,C-BARB AND CONLEY PROMOTIONAL,HOUSTON,TX,77057,BBCN BANK,CA,8-Feb-06,47,5,...,2009,6.0,1,2006,2.0,28,2006,2,8,Wholesale_trade


check


CHGOFF    4884
Name: MIS_Status, dtype: int64

In [22]:
# 47筆改成PIF
# 剩下維持CHGOFF但是要補金額 (先補中位數)
df_pif_weird_chg = df_pif_weird[df_pif_weird['MIS_Status'] == 'CHGOFF']
print('weird chgoff shape: ', df_pif_weird_chg.shape)
print('weird chgoff chgoffdate nullnum: ', df_pif_weird_chg.ChgOffDate.isnull().sum())

# 改標籤
change_to_pif = df_pif_weird_chg[pd.isnull(df_pif_weird_chg['ChgOffDate'])]

df_copy.loc[list(change_to_pif.index), 'MIS_Status'] = 'P I F'

print('Changed:', len(change_to_pif.index), 'rows')

print('=' * 10)
print('check')
df_copy.loc[list(change_to_pif.index), 'MIS_Status'].value_counts()

weird chgoff shape:  (687, 35)
weird chgoff chgoffdate nullnum:  47
Changed: 47 rows
check


P I F    47
Name: MIS_Status, dtype: int64

In [23]:
# 剩下待處理，驗證用
df_pif_weird_PrNull = df_pif_weird_chg[~pd.isnull(df_pif_weird_chg['ChgOffDate'])]
print('weird shape: ', df_pif_weird.shape)

weird shape:  (5571, 35)


In [24]:
# 接下來要把['ChgOffPrinGr'] == 0的排除取中位數填補到 weird_PrNull
med = np.median(df_copy[(df_copy['ChgOffPrinGr'] != 0) & (pd.isnull(df_copy['ChgOffPrinGr']) == False)]['ChgOffPrinGr'])
print('median:', med)

df_copy.loc[list(df_pif_weird_PrNull.index), 'ChgOffPrinGr'] = med
print('Changed:', len(list(df_pif_weird_PrNull.index)), 'rows')

# check

print('=' * 10)
print('check')
df_copy.loc[list(df_pif_weird_PrNull.index), 'ChgOffPrinGr'].value_counts()

median: 32455.5
Changed: 640 rows
check


32455.5    640
Name: ChgOffPrinGr, dtype: int64

In [25]:
print(df_copy.ChgOffDate.isnull().sum())
print(len(df_copy[df_copy['MIS_Status'] == 'P I F']))
# 有 1432 個 CHGOFF 沒有 ChgOffDate
diff = df_copy.ChgOffDate.isnull().sum() - len(df_copy[df_copy['MIS_Status'] == 'P I F'])
total_CHGOFF_num = len(df_copy[df_copy['MIS_Status'] == 'CHGOFF'])

print('diff:', diff)  # 有ChgOffPrinGr且為 CHGOFF 但沒有日期
print('No Date CHGOFF Ratio: {:.2f} %'.format((diff / total_CHGOFF_num) * 100))

736465
735033
diff: 1432
No Date CHGOFF Ratio: 0.87 %


MIS 相關處理結束

In [26]:
# 想用 NoEmp來看是否新創(新創通常人數較少)
df_newexist_miss = df_copy[is_na_or_not(df_copy['NewExist'])]  # 是0 很棒
print('NoEmp Null num:', df_newexist_miss['NoEmp'].isnull().sum())
display(df_newexist_miss['NoEmp'].describe())

# 上網查詢 發現新創公司平均員工人數約20人
new_increase = 0
exist_increase = 0

for i in list(df_newexist_miss.index):
    if df_copy.loc[i, 'NoEmp'] >= 20:
        df_copy.loc[i, 'NewExist'] = 1
        exist_increase += 1
    else:
        df_copy.loc[i, 'NewExist'] = 0
        new_increase += 1

print('null num after operation:', df_copy['NewExist'].isnull().sum())
print('exist increase:', exist_increase)
print('new increase:', new_increase)

NoEmp Null num: 0


count    1170.000000
mean       20.260684
std       151.839915
min         0.000000
25%         3.000000
50%         7.000000
75%        17.000000
max      5000.000000
Name: NoEmp, dtype: float64

null num after operation: 0
exist increase: 269
new increase: 901


In [28]:
# Franchise code太多類別，把它變成一個二元特徵(有無franchise)
display(df_copy['FranchiseCode'].value_counts())
df_copy['FranchiseCode'] = df_copy['FranchiseCode'].replace(1, 0)
df_copy['FranchiseCode'] = df_copy['FranchiseCode'].map({0: 0})  # 不是0的變NA
display(df_copy['FranchiseCode'].value_counts())
df_copy['FranchiseCode'].fillna(1, inplace=True)

df_copy.rename(columns={'FranchiseCode': 'IsFranchised'}, inplace=True)
df_copy['IsFranchised'].value_counts()

1        638554
0        208835
78760      3373
68020      1921
50564      1034
          ...  
17984         1
77173         1
76149         1
75115         1
87344         1
Name: FranchiseCode, Length: 2768, dtype: int64

0.0    847389
Name: FranchiseCode, dtype: int64

0.0    847389
1.0     51775
Name: IsFranchised, dtype: int64

In [31]:
na_check(df_copy)

,Missing Ratio
ChgOffDate,0.819055
ChgOffDate_year,0.819055
ChgOffDate_month,0.819055
ChgOffDate_day,0.819055
DisbursementDate,0.002634
DisbursementDate_year,0.002634
DisbursementDate_month,0.002634
DisbursementDate_day,0.002634
BankState,0.001742
Bank,0.001734


銀行 缺失值處理

In [36]:
df_bankstate_miss = df_copy[is_na_or_not(df_copy['BankState'])]
print(df_bankstate_miss['BankState'].isnull().sum())
df_bankstate_miss

1566


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
407,1003113008,MOUNTVILLE WALL PAPER CO,MOUNTVILLE,PA,17554,NaN,NaN,27-Jun-80,120,57,...,1990,6.0,4,1980,7.0,25,1980,6,27,Unknown
611,1004503009,SMITHFIELD GLASS INC,DANIELSON,CT,6239,NaN,NaN,7-Jul-80,180,30,...,1990,5.0,22,NaN,NaN,NaN,1980,7,7,Unknown
680,1004933009,TAYLOR BROTHERS COMPANY,CLEVELAND,OH,44114,NaN,NaN,8-Jul-80,120,18,...,1989,2.0,8,1980,8.0,21,1980,7,8,Unknown
740,1005273003,"GENERAL SOLAR COMPANY, INC.",SIOUX CITY,IA,51111,NaN,NaN,3-Jul-80,120,3,...,1990,2.0,8,NaN,NaN,NaN,1980,7,3,Unknown
6569,1044653002,SON'S PARTY STORE,KENTWOOD,MI,49508,NaN,NaN,11-Sep-80,180,1,...,1993,10.0,27,NaN,NaN,NaN,1980,9,11,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888537,9772941008,INTERNATIONAL PREHEATER COMPAN,SEATTLE,WA,98188,NaN,NaN,14-Mar-79,84,5,...,1993,12.0,9,NaN,NaN,NaN,1979,3,14,Unknown
888681,9775961009,MANDARIN INTERNATIONAL CO INC,SEATTLE,WA,98104,NaN,NaN,22-Nov-79,24,1,...,1989,4.0,8,1980,1.0,11,1979,11,22,Unknown
897114,9950351001,CLUB 191,PORTSMOUTH,IA,51565,NaN,NaN,24-Jul-78,61,3,...,1988,11.0,23,1978,12.0,6,1978,7,24,Unknown
897474,9958533004,"ROMES CONSTRUCTION, INC.",FORT WAYNE,IN,46804,NaN,NaN,12-Feb-97,3,1,...,NaN,NaN,NaN,1997,4.0,30,1997,2,12,Construction


In [43]:
print(list(set(df_bankstate_miss[~(pd.isnull(df_bankstate_miss)['Bank'])]['Bank'])))
df_bankstate_miss[~(pd.isnull(df_bankstate_miss)['Bank'])]

['STANDARD CHARTERED BK KOREA LT', 'KOREA EXCHANGE BANK', 'THE BANK OF EAST ASIA, LIMITED', 'HONGKONG & SHANGHAI BK. CORP']


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
145106,2060334001,"IBA AND KANG TRADING, INC",NEW YORK,NY,10001,KOREA EXCHANGE BANK,NaN,15-May-98,36,2,...,NaN,NaN,NaN,1998,7.0,31,1998,5,15,Wholesale_trade
424311,4102493005,"CHASE INTERNATIONAL, INC.",CHICAGO,IL,60609,STANDARD CHARTERED BK KOREA LT,NaN,21-May-90,84,16,...,NaN,NaN,NaN,1990,10.0,31,1990,5,21,Manufacturing
444679,4317873009,LIBERTY RESEARCH GROUP INC,GREAT FALLS,MT,59401,HONGKONG & SHANGHAI BK. CORP,NaN,5-Dec-90,31,4,...,NaN,NaN,NaN,1991,1.0,31,1990,12,5,Unknown
447971,4346453000,"ACME MFG. SPECIALTY, INC.",ELK GROVE VILLAGE,IL,60007,STANDARD CHARTERED BK KOREA LT,NaN,15-Jan-91,59,17,...,1993,12.0,1,1991,4.0,30,1991,1,15,Unknown
674674,6910891001,SOMIS SAND & ROCK,SAN DIMAS,CA,91773,HONGKONG & SHANGHAI BK. CORP,NaN,14-Dec-73,84,4,...,1989,1.0,13,1974,1.0,25,1973,12,14,Unknown
793950,8462443003,ATEN-RA INT. GOURMET DELI,NEW YORK,NY,10002,"THE BANK OF EAST ASIA, LIMITED",NaN,27-Jun-95,60,3,...,NaN,NaN,NaN,1995,10.0,31,1995,6,27,Unknown
859233,9236363010,"SACO INTERNATIONAL, INC.",NEW YORK,NY,10017,"THE BANK OF EAST ASIA, LIMITED",NaN,30-Apr-96,12,6,...,NaN,NaN,NaN,1996,5.0,24,1996,4,30,Wholesale_trade


In [45]:
pd.DataFrame(df_copy.groupby(by=['Bank']).size()).loc[list(set(df_bankstate_miss[~(pd.isnull(df_bankstate_miss)['Bank'])]['Bank'])), :]

,0
Bank,
STANDARD CHARTERED BK KOREA LT,2
KOREA EXCHANGE BANK,1
"THE BANK OF EAST ASIA, LIMITED",2
HONGKONG & SHANGHAI BK. CORP,2


In [61]:
a = df_copy.groupby(by=['Bank']).size().sort_values(ascending=False).reset_index()

for i in list(a.Bank):
    if 'CHARTER' in i:
        print(i)

print('=' * 15)

for i in list(a.Bank):
    if 'EAST ASIA' in i:
        print(i)

print('=' * 15)

for i in list(a.Bank):
    if ('HSBC' in i) or ('HONGKONG' in i):
        print(i)

CHARTERBANK
CHARTER BANK EAU CLAIRE
CHARTER BANK
AMERICAN CHARTERED BANK
CHARTER TRUST COMPANY
CHARTER WEST NATIONAL BANK
STANDARD CHARTERED BK KOREA LT
THE BANK OF EAST ASIA, LIMITED
HSBC BK USA NATL ASSOC
HSBC BUSINESS LOANS, INC
HSBC USA INC
HONGKONG & SHANGHAI BK. CORP
HSBC PAYMENT SERVICES (USA) IN


In [67]:
# Name缺失的處理
df_name_miss = df_copy[is_na_or_not(df_copy['Name'])]
print(df_name_miss.shape)
print(df_name_miss.State.isnull().sum())  # 結果不是一起缺失的！
display(df_name_miss.head(3))

# 填unknown
df_copy['Name'].fillna('Unknown', inplace=True)
print('After operation:', df_copy['Name'].isnull().sum(), 'N/A value')

(14, 35)
0


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
272,1002095005,NaN,Logan,UT,84321,JPMORGAN CHASE BANK NATL ASSOC,IL,17-Feb-98,120,1,...,NaN,NaN,NaN,2000,3.0,31,1998,2,17,Wholesale_trade
409,1003125003,NaN,Cobleskill,NY,12043,KEYBANK NATIONAL ASSOCIATION,OH,11-Sep-98,60,5,...,NaN,NaN,NaN,1999,4.0,30,1998,9,11,Unknown
465,1003575009,NaN,Tarrytown,NY,10591,KEYBANK NATIONAL ASSOCIATION,OH,23-Feb-99,60,4,...,NaN,NaN,NaN,1999,5.0,31,1999,2,23,Prof/Science/Tech


After operation: 0 N/A value


In [64]:
# State缺失處理
df_state_miss = df_copy[is_na_or_not(df_copy['State'])]
df_state_miss

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
35517,1270833006,SO. JERSEY DANCE/MERRYLEES,PENNSVILLE,NaN,8070,NaN,NaN,28-Jul-81,120,1,...,1990,3.0,5,NaN,NaN,NaN,1981,7,28,Unknown
49244,1380800010,TRYON COATS & LEATHER,JOHNSTOWN NY,NaN,0,KEYBANK NATIONAL ASSOCIATION,NY,18-May-66,282,0,...,1990,3.0,29,1966,8.0,16,1966,5,18,Unknown
264664,2850643009,CENTURY 21 PHILLIPS REALTY,CAMERON PARK,NaN,95682,SBA - EDF ENFORCEMENT ACTION,CO,12-Feb-87,240,19,...,NaN,NaN,NaN,1987,6.0,17,1987,2,12,Unknown
306274,3113583009,THE COMPUTER EDGE,"BOX 267, APO AP",NaN,96205,RTC/WESTPORT SAVINGS BANK,CA,21-Jan-88,73,8,...,1993,9.0,18,1988,3.0,16,1988,1,21,Prof/Science/Tech
328526,3280213000,KING'S TRUCK WASH,WICHITA,NaN,67219,SOUTH CENT. KANSAS ECONOMIC DE,KS,5-Aug-88,120,17,...,NaN,NaN,NaN,1989,7.0,12,1988,8,5,Other_no_pub
351072,3445553007,JAMES A. NICHOLS,"1542,TABUK,SAUDI ARABIA",NaN,79925,WELLS FARGO BANK NATL ASSOC,TX,19-May-89,16,1,...,NaN,NaN,NaN,1989,7.0,31,1989,5,19,RE/Rental/Lease
366139,3556993002,"GOLF SHOES UNLIMITED, INC.",PALM BEACH GARDENS,NaN,33410,"BEAL BANK, SSB",TX,19-Oct-89,84,3,...,NaN,NaN,NaN,1990,1.0,31,1989,10,19,Retail_trade
366158,3557093005,CASCO SERVICE,CASCO,NaN,54205,UNION STATE BANK,WI,19-Oct-89,204,8,...,NaN,NaN,NaN,1990,4.0,30,1989,10,19,Unknown
367007,3563473008,P & P TOOL INC,SOMERSET,NaN,54025,FIRST BANK OF BALDWIN,WI,26-Oct-89,240,7,...,NaN,NaN,NaN,1990,1.0,31,1989,10,26,Unknown
379174,3664443009,WILLIAMS JEWELERS,SALT LAKE CITY,NaN,84124,ZIONS FIRST NATIONAL BANK,UT,14-Dec-89,60,4,...,NaN,NaN,NaN,1990,7.0,31,1989,12,14,Retail_trade


In [29]:
display(df_copy.iloc[:5,:18])
display(df_copy.iloc[:5,18:])

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,IsFranchised,UrbanRural,RevLineCr,LowDoc,ChgOffDate
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,28-Feb-97,84,4,0.0,0,0,0.0,-1,0.0,1.0,NaN
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,28-Feb-97,60,2,0.0,0,0,0.0,-1,0.0,1.0,NaN
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,28-Feb-97,180,7,1.0,0,0,0.0,-1,0.0,0.0,NaN
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,28-Feb-97,60,2,1.0,0,0,0.0,-1,0.0,1.0,NaN
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,28-Feb-97,240,14,1.0,7,7,0.0,-1,0.0,0.0,NaN


,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,ChgOffDate_year,ChgOffDate_month,ChgOffDate_day,DisbursementDate_year,DisbursementDate_month,DisbursementDate_day,ApprovalDate_year,ApprovalDate_month,ApprovalDate_day,Industry
0,28-Feb-99,60000,0,P I F,0.0,60000,48000,NaN,NaN,NaN,1999,2.0,28,1997,2,28,Retail_trade
1,31-May-97,40000,0,P I F,0.0,40000,32000,NaN,NaN,NaN,1997,5.0,31,1997,2,28,Accom/Food_serv
2,31-Dec-97,287000,0,P I F,0.0,287000,215250,NaN,NaN,NaN,1997,12.0,31,1997,2,28,Healthcare/Social_assist
3,30-Jun-97,35000,0,P I F,0.0,35000,28000,NaN,NaN,NaN,1997,6.0,30,1997,2,28,Unknown
4,14-May-97,229000,0,P I F,0.0,229000,229000,NaN,NaN,NaN,1997,5.0,14,1997,2,28,Unknown


In [30]:
# # groupby
# temp = df_copy[pd.isnull(df_copy['LowDoc'])]
# display(pd.DataFrame(temp.groupby(by=['City', 'State']).size().sort_values(ascending=False).head(30)))
# # temp2 = df_copy[df_copy['City'] == 'EL PASO']
# # display(temp2['LowDoc'].value_counts())
# temp2